In [1]:
# STEP 1: PREPARE THE DATA 

In [2]:
#IMAGE AUGUMENTATION

from keras.preprocessing.image import ImageDataGenerator

train_path = 'C:/Users/Ashutosh/Desktop/New folder/Celebrity-faces/data/train'
test_path = 'C:/Users/Ashutosh/Desktop/New folder/Celebrity-faces/data/val'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   rotation_range=45,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   zoom_range = 0.2,
                                   fill_mode='nearest',
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Using TensorFlow backend.
C:\Users\Ashutosh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ashutosh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ashutosh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ashutosh\anaconda3\lib\site-packages\tensorflow\python\framewo

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [3]:
#STEP 2: MODIFY THE PRE-TRAINED MODEL

In [4]:
# import the pre trained architecture
from keras.applications.vgg16 import VGG16

In [5]:
# load the model withut its top layers

model = VGG16(weights = 'imagenet', 
              include_top = False,
              input_shape = (224,224, 3))

In [6]:
# set all the layers untrainable
# we dont want to change the weights

for layer in model.layers:
    layer.trainable = False

In [7]:
# print all the layer name 

for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [8]:
#STEP 3: PREPARE THE TOP LAYERS OF FINAL MODEL

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D


In [10]:
model_top=model.output
model_top = GlobalAveragePooling2D()(model_top)
model_top = Dense(1024,activation='relu')(model_top)
model_top = Dense(1024,activation='relu')(model_top)
model_top = Dense(512,activation='relu')(model_top)
model_top = Dense(5,activation='softmax')(model_top)

In [11]:
# STEP 4 : MERGE BOTH THE PARTS FOE FINAL MODEL

In [12]:
from keras.models import Model
model_final=Model(inputs= model.input, outputs = model_top)

In [13]:
model_final.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
#STEP 5: MODEL FITTING

In [16]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face_recog.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]


# We use a very small learning rate 
model_final.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 93
nb_validation_samples = 25

# We only train 5 EPOCHS 
epochs = 8
batch_size = 15

history = model_final.fit_generator(
    training_set,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_set,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/8
6/6 [==============================] - 56s 9s/step - loss: 2.8015 - accuracy: 0.3548 - val_loss: 1.1574 - val_accuracy: 0.4800

Epoch 00001: val_loss improved from inf to 1.15740, saving model to face_recog.h5
Epoch 2/8
6/6 [==============================] - 59s 10s/step - loss: 1.2353 - accuracy: 0.5269 - val_loss: 1.1276 - val_accuracy: 0.6000

Epoch 00002: val_loss improved from 1.15740 to 1.12759, saving model to face_recog.h5
Epoch 3/8
6/6 [==============================] - 59s 10s/step - loss: 0.9668 - accuracy: 0.6505 - val_loss: 1.3974 - val_accuracy: 0.4000

Epoch 00003: val_loss did not improve from 1.12759
Epoch 4/8
6/6 [==============================] - 59s 10s/step - loss: 0.9168 - accuracy: 0.6720 - val_loss: 0.8116 - val_accuracy: 0.6800

Epoch 00004: val_loss improved from 1.12759 to 0.81158, saving model to face_recog.h5
Epoch 5/8
6/6 [==============================] - 59s 10s/step - loss: 0.7056 - accuracy: 0.7097 - val_loss: 1.1791 - val_accuracy: 0.5600

E

In [ ]:
# STEP 6: LOAD THE MODEL 

In [17]:
from keras.models import load_model

classifier = load_model('face_recog.h5')

In [18]:
# STEP 7: VIEW THE RESULT

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

celeb_dict = {"[0]": 'ben_afflek', 
              "[1]": 'elton_john',
              "[2]": 'jerry_seinfeld',
              "[3]": 'madonna',
              "[4]": 'mindy_kaling'   }

celeb_dict_n = {"n0": 'ben_afflek', 
                "n1": 'elton_john',
                "n2": 'jerry_seinfeld',
                "n3": 'madonna',
                "n4": 'mindy_kaling'   }

def draw_test(name, pred, im):
    celeb = celeb_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, celeb, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    #print("Class - " + celeb_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Celebrity-faces/data/val/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()